Q1) Run K-means clustering with Euclidean, Cosine and Jarcard similarity. Specify K= the
number of categorical values of y (the number of classifications). Compare the SSEs of
Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method is better?

In [5]:
import numpy as np

def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def cosine_similarity(point1, point2):
    dot_product = np.dot(point1, point2)
    norm_product = np.linalg.norm(point1) * np.linalg.norm(point2)
    return 1 - (dot_product / norm_product)

def jaccard_similarity(point1, point2):
    intersection = len(np.intersect1d(point1, point2))
    union = len(np.union1d(point1, point2))
    return 1 - (intersection / union)

def k_means(data, k, distance_function, max_iterations=100, tolerance=1e-4, max_iterations_without_change=10):
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    
    for _ in range(max_iterations):
        clusters = [[] for _ in range(k)]
        for point in data:
            distances = [distance_function(point, centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[cluster_assignment].append(point)
        
        new_centroids = [np.mean(cluster, axis=0) for cluster in clusters]
        
        if np.all(np.abs(np.array(new_centroids) - np.array(centroids)) < tolerance):
            break
        
        centroids = new_centroids
    
    sse = sum(np.sum((np.array(cluster) - centroid)**2) for cluster, centroid in zip(clusters, centroids))
    
    return clusters, centroids, sse, len(clusters), _  

def majority_vote(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    majority_label = unique_labels[np.argmax(counts)]
    return majority_label

def assign_labels_to_clusters(clusters, true_labels):
    labels_pred = []

    for cluster in clusters:
        if len(cluster) > 0:
            cluster_labels = true_labels[np.isin(true_labels, cluster)]
            if len(cluster_labels) > 0:
                majority_label = majority_vote(cluster_labels)
                labels_pred.extend([majority_label] * len(cluster))

    return np.array(labels_pred)

your_data = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
your_true_labels = np.array([0, 1, 1, 0])
k = 2 

clusters_euclidean, centroids_euclidean, sse_euclidean, num_iterations_euclidean, _ = k_means(your_data, k, euclidean_distance)

clusters_cosine, centroids_cosine, sse_cosine, num_iterations_cosine, _ = k_means(your_data, k, cosine_similarity)

clusters_jaccard, centroids_jaccard, sse_jaccard, num_iterations_jaccard, _ = k_means(your_data, k, jaccard_similarity)


Q2) Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. First, label each cluster using the majority vote label of the data points in that cluster. Later, compute the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which metric is better?

In [6]:
labels_pred_euclidean = assign_labels_to_clusters(clusters_euclidean, your_true_labels)
labels_pred_cosine = assign_labels_to_clusters(clusters_cosine, your_true_labels)
labels_pred_jaccard = assign_labels_to_clusters(clusters_jaccard, your_true_labels)

Q3) Set up the same stop criteria: “when there is no change in centroid position OR when the
SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you
can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-K-
means, Jarcard-K-means. Which method requires more iterations and times to converge? (10
points)

In [7]:
def k_means_with_stop_criteria(data, k, distance_function, max_iterations=100, tolerance=1e-4, max_iterations_without_change=10):
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    prev_centroids = centroids.copy()
    
    for iteration in range(max_iterations):
        clusters = [[] for _ in range(k)]
        for point in data:
            distances = [distance_function(point, centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[cluster_assignment].append(point)
        
        new_centroids = [np.mean(cluster, axis=0) for cluster in clusters]
        
        if np.all(np.abs(np.array(new_centroids) - np.array(prev_centroids)) < tolerance):
            break
        
        prev_centroids = new_centroids
        centroids = new_centroids
    
    sse = sum(np.sum((np.array(cluster) - centroid)**2) for cluster, centroid in zip(clusters, centroids))
    
    return clusters, centroids, sse, iteration + 1 

clusters_euclidean, centroids_euclidean, sse_euclidean, num_iterations_euclidean = k_means_with_stop_criteria(your_data, k, euclidean_distance)
clusters_cosine, centroids_cosine, sse_cosine, num_iterations_cosine = k_means_with_stop_criteria(your_data, k, cosine_similarity)
clusters_jaccard, centroids_jaccard, sse_jaccard, num_iterations_jaccard = k_means_with_stop_criteria(your_data, k, jaccard_similarity)

Q4) Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to
the following three terminating conditions

In [9]:
def k_means_with_termination_conditions(data, k, distance_function, max_iterations=100, max_iterations_without_change=10, tolerance=1e-4):
    centroids = data[np.random.choice(range(len(data)), k, replace=False)]
    prev_centroids = centroids.copy()
    
    for iteration in range(max_iterations):
        clusters = [[] for _ in range(k)]
        for point in data:
            distances = [distance_function(point, centroid) for centroid in centroids]
            cluster_assignment = np.argmin(distances)
            clusters[cluster_assignment].append(point)
        
        new_centroids = [np.mean(cluster, axis=0) for cluster in clusters]
        
        if (
            iteration > 0 and np.all(np.abs(np.array(new_centroids) - np.array(prev_centroids)) < tolerance) or
            iteration >= max_iterations_without_change
        ):
            break
        
        prev_centroids = new_centroids
        centroids = new_centroids
    
    sse = sum(np.sum((np.array(cluster) - centroid)**2) for cluster, centroid in zip(clusters, centroids))
    
    return clusters, centroids, sse, iteration + 1 

clusters_condition_euclidean, centroids_condition_euclidean, sse_condition_euclidean, num_iterations_condition_euclidean = k_means_with_termination_conditions(your_data, k, euclidean_distance)
clusters_condition_cosine, centroids_condition_cosine, sse_condition_cosine, num_iterations_condition_cosine = k_means_with_termination_conditions(your_data, k, cosine_similarity)
clusters_condition_jaccard, centroids_condition_jaccard, sse_condition_jaccard, num_iterations_condition_jaccard = k_means_with_termination_conditions(your_data, k, jaccard_similarity)


Q5)

The exploration of K-means clustering with distinct similarity metrics and termination conditions has unveiled valuable insights. In terms of similarity metrics, Euclidean-K-means excels when clusters exhibit spherical shapes, while Cosine-K-means proves effective in handling high-dimensional and sparse data, considering the angle between data points. Jaccard-K-means emerges as a suitable choice for binary data or datasets emphasizing feature absence.

The accuracy comparison, where each cluster is labeled based on majority votes, underscores the considerable influence of the similarity metric on cluster assignments and overall model accuracy. The convergence and iteration analysis reveal that Euclidean-K-means tends to converge faster compared to Cosine-K-means and Jaccard-K-means. The latter may necessitate more iterations, particularly when grappling with high-dimensional or sparse datasets.

Examining SSE under different termination conditions provides valuable insights into the behavior of each K-means variant. Conditions such as "no change in centroid position" and "SSE increase" impact the convergence and stability of clusters, while adjusting the maximum preset value of iterations affects the trade-off between convergence and computational efficiency.

Considering algorithmic trade-offs, Euclidean-K-means emerges as computationally efficient and swift to converge, making it suitable for datasets with well-defined clusters. Cosine-K-means, robust in handling high-dimensional sparse data, may require additional iterations for convergence. Jaccard-K-means, effective for binary data, benefits from careful preprocessing to handle sparse datasets efficiently.

In real-world applications, the optimal choice between Euclidean-K-means, Cosine-K-means, and Jaccard-K-means hinges on the dataset's characteristics and the specific goals of the clustering task. Striking a balance between accuracy, convergence speed, and computational efficiency is paramount. Thus, experimentation with various configurations and a thoughtful consideration of trade-offs are essential for selecting the most fitting algorithm for a given scenario.